In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

Running on Apache Spark version 2.4.3
SparkUI available at http://p1756.prometheus:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/hail-20200908-2056-0.2.30-2ae07d872f43.log


Loading BokehJS ...

Loading BokehJS ...

### note - paths in this notebook are outdated

In [2]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/*gz

In [17]:
names = []

for i in files:
    i = i.split('/')
    i = i[9]
    i = i.split('.')
    i = i[0]
    names.append(i)

In [ ]:
#importing part - 10

small_mts = []

small = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/part10/vcf-part-10-17-to-20/*gz

for f in small:
    mt = hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False)
    print(mt.count())
    small_mts.append(mt)
    
mt_small = hl.MatrixTable.union_rows(*small_mts)

In [ ]:
mt_small.count()

In [ ]:
larger_mts = []

larger = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-vcfs/part10/*gz

for f in larger:
    mt = hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False)
    print(mt.count())
    larger_mts.append(mt)
    
mt_larger = hl.MatrixTable.union_rows(*larger_mts)

In [11]:
mt_larger.count()

2020-08-01 09:11:42 Hail: INFO: Coerced sorted dataset
2020-08-01 09:11:45 Hail: INFO: Coerced sorted dataset
2020-08-01 09:11:47 Hail: INFO: Coerced sorted dataset
2020-08-01 09:11:51 Hail: INFO: Coerced sorted dataset
2020-08-01 09:11:55 Hail: INFO: Coerced sorted dataset
2020-08-01 09:11:58 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:02 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:05 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:08 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:12 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:16 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:20 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:23 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:26 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:29 Hail: INFO: Coerced sorted dataset
2020-08-01 09:12:33 Hail: INFO: Coerced sorted dataset


(1240271, 186)

In [15]:
mt_larger = mt_larger.filter_cols(mt_larger.s != '464')

In [16]:
mt_10 = mt_larger.union_rows(mt_small)

2020-08-01 09:17:31 Hail: INFO: Coerced sorted dataset
2020-08-01 09:17:31 Hail: INFO: Coerced sorted dataset


In [ ]:
mt_10.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part010.mt')

In [52]:
for idx, f in enumerate(files):
    hl.import_vcf(f, reference_genome='GRCh38', filter='chr9[ \t]138334443', force_bgz = True, array_elements_required = False).write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/'+names[idx]+'.mt')

2020-07-28 11:45:43 Hail: INFO: Coerced sorted dataset
2020-07-28 11:45:53 Hail: INFO: wrote matrix table with 1683295 rows and 186 columns in 80 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part001.mt
2020-07-28 11:45:57 Hail: INFO: Coerced sorted dataset
2020-07-28 11:46:06 Hail: INFO: wrote matrix table with 1558301 rows and 186 columns in 77 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part002.mt
2020-07-28 11:46:10 Hail: INFO: Coerced sorted dataset
2020-07-28 11:46:21 Hail: INFO: wrote matrix table with 1649527 rows and 186 columns in 78 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part003.mt
2020-07-28 11:46:27 Hail: INFO: Coerced sorted dataset
2020-07-28 11:46:39 Hail: INFO: wrote matrix table with 1698695 rows and 186 columns in 82 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part004.mt
2020-07-28 11:46:43 Hail: INFO: Coerced sorted dataset
2020-07-2

In [70]:
mts = []
for i in names:
    i = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/'+i+'.mt')
    mts.append(i)

In [64]:
for mt in mts:
    pprint(mt.aggregate_rows(hl.agg.counter(mt.locus.contig)))

{'chr1': 1683295}
{'chr1': 931440, 'chr2': 626861}
{'chr2': 1649527}
{'chr2': 287629, 'chr3': 1411066}
{'chr3': 768941, 'chr4': 850832}
{'chr4': 1066702, 'chr5': 424812}
{'chr5': 1441040}
{'chr6': 1582520}
{'chr7': 1582770}
{'chr10': 223031, 'chr9': 1253669}
{'chr10': 1226568}
{'chr11': 988270, 'chr12': 720672}
{'chr12': 713356, 'chr13': 783296}
{'chr13': 294127, 'chr14': 984923, 'chr15': 2601}
{'chr15': 917493, 'chr16': 639679}
{'chr16': 250054, 'chr17': 857844, 'chr18': 525502}
{'chr18': 313038, 'chr19': 621905, 'chr20': 734413}
{'chr21': 401883, 'chr22': 428519, 'chrX': 592378}
{'chrX': 952330, 'chrY': 103650}


In [19]:
rpmk = hl.import_bed('repeatmasker_all', reference_genome='GRCh38', skip_invalid_intervals=True)

2020-08-01 09:22:22 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)



In [20]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-mts/part010.mt')

In [21]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/part010.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/part010.mt')

2020-08-01 09:22:33 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-01 09:22:49 Hail: INFO: wrote matrix table with 768173 rows and 185 columns in 228 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/part010.mt


In [ ]:
for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-rpmk/'+names[idx]+'.mt')

In [72]:
for mt in mts:
    print(mt.count())

(815382, 186)
(885602, 186)
(926109, 186)
(952778, 186)
(859760, 186)
(811815, 186)
(800986, 186)
(867720, 186)
(848696, 186)
(792678, 186)
(664390, 186)
(868098, 186)
(736343, 186)
(741541, 186)
(868784, 186)
(869465, 186)
(836089, 186)
(708449, 186)
(476900, 186)


In [22]:
table = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/gnomad.genomes.r3.0.coverage.summary.tsv.bgz', impute = True)

table = table.select(table.locus, table.over_1)

table = table.annotate(locus = hl.parse_locus(table.locus, reference_genome='GRCh38'))

table = table.filter(table.over_1 >0.9)

table = table.key_by(table.locus)

2020-08-01 09:22:49 Hail: INFO: Reading table to impute column types
2020-08-01 09:25:43 Hail: INFO: Finished type imputation
  Loading column 'locus' as type 'str' (imputed)
  Loading column 'mean' as type 'float64' (imputed)
  Loading column 'median' as type 'int32' (imputed)
  Loading column 'over_1' as type 'float64' (imputed)
  Loading column 'over_5' as type 'float64' (imputed)
  Loading column 'over_10' as type 'float64' (imputed)
  Loading column 'over_15' as type 'float64' (imputed)
  Loading column 'over_20' as type 'float64' (imputed)
  Loading column 'over_25' as type 'float64' (imputed)
  Loading column 'over_30' as type 'float64' (imputed)
  Loading column 'over_50' as type 'float64' (imputed)
  Loading column 'over_100' as type 'float64' (imputed)


In [27]:
mt = mt.filter_rows(hl.is_defined(table[mt.locus]), keep = True)
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')

2020-08-01 09:29:44 Hail: INFO: Coerced sorted dataset
2020-08-01 09:30:19 Hail: INFO: wrote matrix table with 735860 rows and 185 columns in 228 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt


In [ ]:
for idx, mt in enumerate(mts):
    print(mt.count())
    mt = mt.filter_rows(hl.is_defined(table[mt.locus]), keep = True)
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+names[idx]+'.mt')
    mts[idx] = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+names[idx]+'.mt')

(815382, 186)


2020-07-28 12:20:01 Hail: INFO: Coerced sorted dataset
2020-07-28 12:20:27 Hail: INFO: wrote matrix table with 778366 rows and 186 columns in 80 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part001.mt


(885602, 186)


2020-07-28 12:24:36 Hail: INFO: Coerced sorted dataset
2020-07-28 12:25:04 Hail: INFO: wrote matrix table with 862162 rows and 186 columns in 77 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part002.mt


(926109, 186)


2020-07-28 12:29:16 Hail: INFO: Coerced sorted dataset
2020-07-28 12:29:44 Hail: INFO: wrote matrix table with 904404 rows and 186 columns in 78 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part003.mt


(952778, 186)


## Start from here 

In [27]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/

In [3]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/'+f)
    mts.append(f)

In [28]:
gnomad = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/gnomad/gnomad_selected_filtered.ht')

In [31]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-cov/part010.mt')
mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
mt = hl.split_multi_hts(mt)
mt = mt.annotate_rows(gnomad_v3 = gnomad[mt.row_key])
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part010.mt')

2020-08-01 10:12:09 Hail: INFO: wrote matrix table with 772748 rows and 185 columns in 228 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part010.mt


In [12]:
for idx, mt in enumerate(mts):
    mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
    mt = hl.split_multi_hts(mt)
    mt = mt.annotate_rows(gnomad_v3 = gnomad[mt.row_key])
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/'+files[idx])

2020-07-29 17:50:12 Hail: INFO: wrote matrix table with 812793 rows and 186 columns in 80 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part001.mt
2020-07-29 17:50:22 Hail: INFO: wrote matrix table with 902733 rows and 186 columns in 77 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part002.mt
2020-07-29 17:50:33 Hail: INFO: wrote matrix table with 942334 rows and 186 columns in 78 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part003.mt
2020-07-29 17:50:44 Hail: INFO: wrote matrix table with 971532 rows and 186 columns in 82 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part004.mt
2020-07-29 17:50:53 Hail: INFO: wrote matrix table with 872753 rows and 186 columns in 79 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part005.mt
2020-07-29 17:51:06 Hail: INFO: wrote matrix table with 824928 rows and 186 columns in 74 parti

Now - see example annotations and annotate with nearby genes and HPO

In [57]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/'+f)
    mts.append(f)

In [32]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/genecode_v32.ht')

In [33]:
genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

In [34]:
hpo = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing/hpo.tsv', impute = True, no_header=True)

2020-08-01 10:15:18 Hail: INFO: Reading table to impute column types
2020-08-01 10:15:19 Hail: INFO: Finished type imputation
  Loading column 'f0' as type 'str' (imputed)
  Loading column 'f1' as type 'str' (imputed)


In [35]:
start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

In [40]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-gnomad/part010.mt')
mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt= mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/part010.mt')

2020-08-01 10:15:30 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-01 10:15:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-01 10:15:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-01 10:15:33 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-08-01 10:15:54 Hail: INFO: wrote matrix table with 772748 rows and 185 columns in 228 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/part010.mt


In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
    mts[idx] = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
    mt.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/'+files[idx])

In [41]:
files = !ls /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/

In [43]:
mts = []

for f in files:
    f = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-hpo/'+f)
    f = f.filter_cols(f.s != '464')
    mts.append(f)

## one last step - join + perform PCA 

In [44]:
mt_full = hl.MatrixTable.union_rows(*mts)

In [45]:
mt_full.write('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt')

2020-08-01 10:17:52 Hail: INFO: wrote matrix table with 16233065 rows and 185 columns in 1604 partitions to /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt


In [38]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS.mt')

In [39]:
mt = mt.annotate_rows(info = hl.struct(AC = mt.AC, AF = mt.AF, gnomad_nfe_AF = mt.gnomad_v3.v3_nfe.AF, gnomad_nfe_homozygote_count = mt.gnomad_v3.v3_nfe.homozygote_count, highest_consequence = mt.gnomad_v3.most_severe_consequence,
                                  exon = mt.gnomad_v3.transcript_consequences.exon,
                                  transcript_impact = mt.gnomad_v3.transcript_consequences.impact, lof = mt.gnomad_v3.transcript_consequences.lof,
                                  pholyphen_prediction = mt.gnomad_v3.transcript_consequences.polyphen_prediction, polyphen_score = mt.gnomad_v3.transcript_consequences.polyphen_score, 
                                  sift_prediction = mt.gnomad_v3.transcript_consequences.sift_prediction, sift_score = mt.gnomad_v3.transcript_consequences.sift_score,
                                  transcript_id = mt.gnomad_v3.transcript_consequences.transcript_id, variant_class = mt.gnomad_v3.variant_class, gene_symbol = mt.within_gene, hpo = hl.flatten(mt.hpo)))

In [40]:
hl.export_vcf(mt, '/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS_all.vcf.bgz')

2020-08-02 20:10:24 Hail: WARN: export_vcf: ignored the following fields:
    'AC' (row)
    'AF' (row)
    'AN' (row)
    'a_index' (row)
    'was_split' (row)
    'gnomad_v3' (row)
    'within_gene' (row)
    'hpo' (row)
2020-08-02 20:11:48 Hail: INFO: merging 1604 files totalling 13.4G...
2020-08-02 20:13:23 Hail: INFO: while writing:
    /net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS_all.vcf.bgz
  merge time: 1m35.5s


In [41]:
samples = mt.s.collect()

In [ ]:
for s in samples:
    mt_s = mt.filter_cols(mt.s == s)
    hl.export_vcf(mt_s,'/net/archive/groups/plggneuromol/GTS-analysis/data/vcf/GTS-annotated-vcf/GTS-individual-vcf'+s+'vcf.bgz')

In [48]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_sample.GT)
mt_sample = mt_sample.annotate_cols(scores = pcs[mt_sample.s].scores)

p = hl.plot.scatter(mt_sample.scores[0],
                    mt_sample.scores[1],
                    label=mt_sample.s,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

2020-08-01 10:18:21 Hail: INFO: hwe_normalized_pca: running PCA using 8109 variants.
2020-08-01 10:18:48 Hail: INFO: pca: running PCA with 10 components...


In [49]:
mt_sample = mt_sample.annotate_cols(group = hl.cond(mt_sample.s.contains("S"), "old", "new"))

In [53]:
p = hl.plot.scatter(mt_sample.scores[3],
                    mt_sample.scores[4],
                    label=mt_sample.group,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [57]:
pwd

'/net/archive/groups/plggneuromol/GTS-analysis/vcf_preprocessing'